# Segmentação de imagens

Técnicas para identificação de objetos e formas.

In [139]:
import cv2
print(cv2.__version__)

import numpy as np

from matplotlib import pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline

plt.style.use('seaborn')

3.3.1


Verificando versão instalada do OpenCV

## Correspondência de formas (*shape matching*)

In [356]:
image_contour_approx = image.copy()

for contour in contours:
    convex_hull = cv2.convexHull(contour)
    area = cv2.contourArea(contour)
    print(area)
    if area > 100:
        cv2.drawContours(image_contour_approx, [convex_hull], 0, (0, 255, 0), 2)

cv2.imshow("Hand Hull", image_contour_approx)
cv2.waitKey()
cv2.destroyAllWindows()

0.5
1.0
3.5
4.0
4.0
4.0
6.0
6.0
8.0
8.0
8.0
10.0
18.0
23.0
26.0
26.0
345.0


A utilização de contornos para identificação de formas também nos abre possibilidades de comparação entre elas. Somente utilizando aspecto geométrico, é possível identificar formas em imagens.

In [2]:
image = cv2.imread("imagens/shapes.jpg")
image_target = cv2.imread("imagens/star.jpg")


cv2.imshow("Fruits", image)
cv2.waitKey()

cv2.imshow("Target", image_target)
cv2.waitKey()
cv2.destroyAllWindows()

In [6]:
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
_, image_binary = cv2.threshold(image_gray, 230, 255,cv2.THRESH_BINARY_INV)

image_gray_target = cv2.cvtColor(image_target, cv2.COLOR_BGR2GRAY)
_, image_target_binary = cv2.threshold(image_gray_target, 230, 255,cv2.THRESH_BINARY_INV)


cv2.imshow("Fruits Binary", image_binary)
cv2.waitKey()

cv2.imshow("Target Binary", image_target_binary)
cv2.waitKey()
cv2.destroyAllWindows()

In [7]:
image_target_contour = image_target.copy()
_, contour_target, hierarchy = cv2.findContours(image_target_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(image_target_contour, contour_target, 0, (0, 255, 0), 2)

cv2.imshow("Target Contour", image_target_contour)
cv2.waitKey()
cv2.destroyAllWindows()

In [8]:
image_detected = image.copy()

cv2.imshow("Detected B", image_binary)
cv2.waitKey()
cv2.destroyAllWindows()

_, contours, hierarchy = cv2.findContours(image_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    #Quanto menor o valor mais similaridade existe
    match_perc = cv2.matchShapes(contour_target[0], contour, 1, 0)
    print(match_perc)
    if match_perc < 0.05:
        match_contour = contour   
        cv2.drawContours(image_detected, [match_contour], 0, (0, 255, 0), 2)
        
cv2.imshow("Detected", image_detected)
cv2.waitKey()
cv2.destroyAllWindows()

0.21697076862173215
0.10479351156122929
0.05397178326730856
0.24908817898612212
0.027009941605979915
0.23740218637769916


## Detecção de linhas

In [62]:
image = cv2.imread("imagens/campo.png")

cv2.imshow("Campo Futebol", image)
cv2.waitKey()
cv2.destroyAllWindows()

In [67]:
image_gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

image_edges = cv2.Canny(image_gray, 100, 250, apertureSize = 3)

cv2.imshow("Campo Futebol Edges", image_edges)
cv2.waitKey()
cv2.destroyAllWindows()

image_detecada = image.copy()

lines = cv2.HoughLines(image_edges, 1, np.pi/180, 100)

for line in lines:
    print(line)
    for rho, theta in line:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        cv2.line(image_detecada,(x1,y1),(x2,y2),(0,0,255),2)

cv2.imshow("Campo Futebol Lines", image_detecada)
cv2.waitKey()
cv2.destroyAllWindows()

[[-263.           3.0892327]]
[[-260.           3.0892327]]
[[-15.          3.0892327]]
[[-18.          3.0892327]]
[[-265.          3.106686]]
[[-20.         3.106686]]
[[211.          1.5184364]]
[[28.         1.5184364]]
[[395.          1.5184364]]
[[398.          1.5184364]]
[[215.          1.5184364]]
[[30.         1.5184364]]


## Identificação de círculos

In [133]:
image = cv2.imread("imagens/moedas.jpg")

cv2.imshow("Moedas", image)
cv2.waitKey()
cv2.destroyAllWindows()

In [134]:
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image_blur = cv2.medianBlur(image_gray, 5)

circles = cv2.HoughCircles(image_blur, cv2.HOUGH_GRADIENT, 50, 80)
 
for i in circles[0,:]:
    #Círculo em volta da moeda
    cv2.circle(image,(i[0], i[1]), i[2], (255, 0, 0), 2) 
    #Cïrculo no centro da moeda
    cv2.circle(image, (i[0], i[1]), 2, (0, 255, 0), 5)
 
cv2.imshow("Moedas Detectadas", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Deteção de objetos

## Correspondência por *template*

Este tipo de técnica busca por um objeto pré-definido em uma imagem. A forma da busca é como se fosse uma janela que varre a imagem continuamente até o fim.

In [151]:
image = cv2.imread("imagens/mario.jpg")
cv2.imshow("Mario", image)
cv2.waitKey()

image_template = cv2.imread("imagens/coin_mario_2.jpg", cv2.IMREAD_GRAYSCALE)

cv2.imshow("Moeda Template", image_template)
cv2.waitKey()

cv2.destroyAllWindows()

In [224]:
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image_matched = image.copy()

matched_template = cv2.matchTemplate(image_gray, image_template, cv2.TM_CCOEFF_NORMED)
w, h = image_template.shape[::-1]

result = cv2.minMaxLoc(matched_template)
threshold = 0.9
loc = np.where(matched_template >= threshold)

#Índice é a informação da coordenada
for pt in zip(*loc[::-1]):
    cv2.rectangle(image_matched, pt, (pt[0] + w, pt[1] + h), (0,255,0), 1)

cv2.imshow("Moeda Matched", image_matched)
cv2.waitKey()
cv2.destroyAllWindows()

# Revisão zip com tuplas
#zipper_list = [(1, 'a'), (2, 'b'), (3, 'c')]
#list_a, list_b = zip(*zipper_list)
#print list_a # (1, 2, 3)
#print list_b # ('a', 'b', 'c')

In [229]:
image = cv2.imread("imagens/wally.jpg")
cv2.imshow("Wally", image)
cv2.waitKey()

image_template = cv2.imread("imagens/wally_face.jpg", cv2.IMREAD_GRAYSCALE)

cv2.imshow("Wally Template", image_template)
cv2.waitKey()

cv2.destroyAllWindows()

Método para localização de apenas um elemneto. A função **minMaxLoc** irá trazer o objeto com maior valor de correspondência.

In [ ]:
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image_matched = image.copy()

matched_template = cv2.matchTemplate(image_gray, image_template, cv2.TM_CCOEFF_NORMED)
w, h = image_template.shape[::-1]

min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(matched_template)

top_left = max_loc
bottom_right = (top_left[0] + w, top_left[1] + h)

cv2.rectangle(image_matched, top_left, bottom_right, (0,255,0), 2)

cv2.imshow("Moeda Matched", image_matched)
cv2.waitKey()
cv2.destroyAllWindows()

## SIFT

Os algoritmos SIFT e SURF foram patenteados e não estão presentes nas versões oficiais do OpenCV 3.X. Para utilizá-los sem fins lucrativos é preciso instalar um pacote que os habilita, o **opencv-contrib**.

No ambiente do Anaconda utilize o comando abaixo:

**conda install -c michael_wild opencv-contrib**

__https://www.pyimagesearch.com/2015/07/16/where-did-sift-and-surf-go-in-opencv-3/__

In [308]:
image = cv2.imread("imagens/cristo.jpg")
cv2.imshow("Cristo Redentor", image)
cv2.waitKey()
cv2.destroyAllWindows()

In [259]:
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
sift_detector = cv2.xfeatures2d.SIFT_create()
(kps, descs) = sift_detector.detectAndCompute(image_gray, None)
print("Pontos detectados " + str(len(kps)))

Pontos detectados 259


In [258]:
image_detected = image.copy()
image_detected = cv2.drawKeypoints(image_detected, kps, image_detected, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow("Cristo Redentor Discriminant", image_detected)
cv2.waitKey()
cv2.destroyAllWindows()

## SURF

In [278]:
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
surf_detector = cv2.xfeatures2d.SURF_create()
surf_detector.setHessianThreshold(5000)
(kps, descs) = surf_detector.detectAndCompute(image_gray, None)
print("Pontos detectados " + str(len(kps)))

Pontos detectados 69


In [279]:
image_detected = image.copy()
image_detected = cv2.drawKeypoints(image_detected, kps, image_detected, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow("Cristo Redentor Discriminant", image_detected)
cv2.waitKey()
cv2.destroyAllWindows()

## FAST

In [290]:
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
fast_detector = cv2.FastFeatureDetector_create()
kps = fast_detector.detect(image_gray, None)
print("Pontos detectados " + str(len(kps)))

Pontos detectados 1590


In [299]:
image_detected = image.copy()
image_detected = cv2.drawKeypoints(image_detected, kps, image_detected, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow("Cristo Redentor Discriminant", image_detected)
cv2.waitKey()
cv2.destroyAllWindows()

## BRIEF

In [300]:
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
fast_detector = cv2.FastFeatureDetector_create()
brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()
kps = fast_detector.detect(image_gray, None)
kps, desc = brief.compute(gray, kps)
print("Pontos detectados " + str(len(kps)))

Pontos detectados 1586


In [301]:
image_detected = image.copy()
image_detected = cv2.drawKeypoints(image_detected, kps, image_detected, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow("Cristo Redentor Discriminant", image_detected)
cv2.waitKey()
cv2.destroyAllWindows()

## ORB

In [313]:
image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
orb_detector = cv2.ORB_create(200)
kps = orb_detector.detect(image_gray, None)
kps, desc = orb_detector.compute(image_gray, kps)
print("Pontos detectados " + str(len(kps)))

Pontos detectados 200


In [314]:
image_detected = image.copy()
image_detected = cv2.drawKeypoints(image_detected, kps, image_detected, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow("Cristo Redentor Discriminant", image_detected)
cv2.waitKey()
cv2.destroyAllWindows()

## Correspondência dos descritores com outras imagens

## Força bruta

In [340]:
image_target = cv2.imread("imagens/cristo.jpg")
image_target_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

image_search = cv2.imread("imagens/cristo-redentor.jpg")
image_search_gray = cv2.cvtColor(image_search, cv2.COLOR_BGR2GRAY)

orb_detector = cv2.ORB_create(5000)

kps = orb_detector.detect(image_target_gray, None)
kps_target, desc_target = orb_detector.compute(image_target_gray, kps)

kps = orb_detector.detect(image_search_gray, None)
kps_search, desc_search = orb_detector.compute(image_search_gray, kps)

In [342]:
#Correspondência utilizando Hamming
bf = cv2.BFMatcher_create(cv2.NORM_HAMMING, crossCheck=True)

#Correspondência dos descritores
matches = bf.match(desc_target, desc_search)

#Ordenar pela distância dos pontos (similaridaed)
matches = sorted(matches, key = lambda x:x.distance)

#Obter os 100 pontos detectados
image_detected = cv2.drawMatches(image_target_gray, kps_target, image_search_gray, kps_search, matches[:100], None, flags=2)

cv2.imshow("Cristo Redentor Discriminant", image_detected)
cv2.waitKey()
cv2.destroyAllWindows()

## Correspondência por FLAN

In [352]:
FLANN_INDEX_LSH = 6
index_params= dict(algorithm = FLANN_INDEX_LSH, table_number = 6, key_size = 12, multi_probe_level = 1) 

search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

matches = flann.knnMatch(desc_target, desc_search, k=2)

# Need to draw only good matches, so create a mask
matchesMask = [[0,0] for i in range(len(matches))]

# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.7*n.distance:
        matchesMask[i]=[1,0]
        
draw_params = dict(matchColor = (0,255,0), singlePointColor = (255,0,0), matchesMask = matchesMask, flags = 0)
image_detected = cv2.drawMatchesKnn(image_target_gray, kps_target, image_search_gray, kps_search, matches, None, **draw_params)

cv2.imshow("Cristo Redentor Discriminant", image_detected)
cv2.waitKey()
cv2.destroyAllWindows()